# Level 2 - Week 8 - 01 Endpoint Map and Boundaries

**Estimated time:** 60-90 minutes

## Learning Objectives

- List endpoints by role
- Clarify UI vs API responsibilities
- Keep /search for debugging


## Overview

Keep the backend as the source of truth.

UI should:

- call APIs
- render results (including citations)

Backend should:

- implement retrieval/chat/eval logic

## Underlying theory: trust boundaries and sources of truth

### Backend as the policy enforcement point

The backend is where you can enforce invariants:

- grounding and citation rules
- authn/authz
- rate limits and budgets
- consistent evaluation

If business logic is duplicated in UI and API, you create two divergent “truths”.

### UI is an untrusted environment

Even if you wrote the UI:

- users can modify front-end code in the browser
- requests can be replayed with curl/Postman

So any rule that matters must be enforced server-side.

## Practice Steps

- Create an endpoint map (user/admin/system).
- Add ownership notes (UI responsibility vs API responsibility).
- Keep `/search` as a debugging endpoint (protected in prod, but available in dev).

### Sample code

Endpoint map template.


In [ ]:
endpoint_map = {
    'user': ['/chat', '/search'],
    'admin': ['/ingest', '/eval'],
    'system': ['/health', '/ready'],
}

print(endpoint_map)


### Student fill-in

Add notes on which layer owns which logic.


In [ ]:
endpoint_map = {
    "user": ["/chat", "/search"],
    "admin": ["/ingest", "/eval"],
    "system": ["/health", "/ready"],
}

ownership_notes = {
    "/chat": "API owns retrieval+grounding+citation enforcement; UI only renders",
    "/search": "API owns retrieval; expose in dev for attribution/debug",
    "/ingest": "API owns ingestion pipeline; admin-only",
    "/eval": "API owns eval scripts; admin-only",
    "/health": "API system endpoint",
    "/ready": "API dependency checks (optional)",
}

print(endpoint_map)
print(ownership_notes)

## Self-check

- Is retrieval logic only in the backend?
- Is /search still available for debugging?


## Practical usage: why `/search` should remain exposed (at least in dev)

`/chat` is a composite pipeline (retrieve → pack → generate).

When an answer is wrong, `/search` lets you isolate retrieval:

- did retrieval return the right evidence?
- did generation ignore good evidence?

Keep `/search` protected in production, but don’t delete it during development.

### Exercise: Endpoint map audit

For each endpoint, write down:

- who can call it (user/admin)
- what invariant it must enforce
- what you would log (minimum evidence for debugging)

Goal: make it obvious where policy lives (server-side) and how you would debug failures.

audit = {
    "/chat": {"role": "user", "invariants": ["mode in {answer,clarify,refuse}", "citations traceable"], "log": ["request_id", "retrieved_chunk_ids", "mode"]},
    "/search": {"role": "user", "invariants": ["stable hit ordering"], "log": ["request_id", "query", "top_k", "hit_chunk_ids"]},
    "/ingest": {"role": "admin", "invariants": ["idempotent upsert", "stable chunk_ids"], "log": ["request_id", "actor", "resource", "status"]},
    "/eval": {"role": "admin", "invariants": ["fixed eval set", "write artifacts"], "log": ["run_id", "config", "metrics"]},
}

for k, v in audit.items():
    print(k, v)

In [ ]:
# Self-check
#
# - Can you test backend behavior without the UI?
# - Can you swap UIs without changing backend logic?
# - Is retrieval logic enforced server-side (not duplicated in the UI)?
# - Do you have a /search path for debugging attribution?

# (End) Next: implement minimal authn/authz + audit logging for admin endpoints in Part 02.